<a href="https://colab.research.google.com/github/Kareem-Zaghawa/EFG_Task/blob/main/EFG_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import cv2
import numpy as np
import os

def remove_logo(image_path, logo_path, output_path):

    main_image = cv2.imread(image_path)
    logo_image = cv2.imread(logo_path)

    # Convert images to grayscale becasue SIFT detects key points based
    # on intensity changes in the image. Changes in greyscale are easier to capture than across RGB
    # Also less computationally expensive.

    gray_main_image = cv2.cvtColor(main_image, cv2.COLOR_BGR2GRAY)
    gray_logo_image = cv2.cvtColor(logo_image, cv2.COLOR_BGR2GRAY)

    sift = cv2.SIFT_create()

    # Keypoints are certain points on the image that are distinct and can be reliably detected and matched across different images.
    # In our case the key points we want to get is the logo and match them with the keypoints of logo in the ID image.
    #
    # Descriptors are vectors that describe the region around keypoints which will be used to detect matches between the logo and the image.
    kp1, des1 = sift.detectAndCompute(gray_logo_image, None)
    kp2, des2 = sift.detectAndCompute(gray_main_image, None)

    # Match descriptors using FLANN-based matcher
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # Store good matches using Lowe's ratio test. The ideal ratio is from 0.5 to 0.9.
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    MIN_MATCH_COUNT = 10
    if len(good_matches) > MIN_MATCH_COUNT:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 2)

        # Homography is a transformation that maps points from one plane to another.
        # In our case, mapping the corners of the logo image to their coressponfing poistion in the ID image.
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        matches_mask = mask.ravel().tolist()

        h, w, _ = logo_image.shape

        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)

        dst = cv2.perspectiveTransform(pts, M)

        # Draw the bounding box
        dst = np.int32(dst)
        rect = cv2.boundingRect(dst)
        x, y, w, h = rect


        # Calculate the average color of the 4 vertices of the bounding box to draw a rectange of the same color on the logo
        vertices = dst.reshape(-1, 2)
        average_color = np.mean([main_image[int(vertex[1]), int(vertex[0])] for vertex in vertices], axis=0)

        cv2.rectangle(main_image, (x, y), (x + w, y + h), average_color.tolist(), -1)

        cv2.imwrite(output_path, main_image)
    else:
        print(f"Not enough matches are found - {len(good_matches)}/{MIN_MATCH_COUNT}")
        matches_mask = None

# Usage
#main_image_path = '/content/Input/9.jpg'
logo_image_path = '/content/Logos/LOGO3.jpg'
#output_image_path = '/content/Output/2_with_bounding_box.jpg'
#detect_and_hide_logo(main_image_path, logo_image_path, output_image_path)


In [5]:
output_directory = '/content/output'
os.makedirs(output_directory, exist_ok=True)

input_directory = '/content/Input'

# Loop over all files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.jpg'):
        target_image_path = os.path.join(input_directory, filename)
        output_image_path = os.path.join(output_directory,f'{os.path.splitext(filename)[0] }_with_bounding_box.jpg')
        remove_logo(target_image_path,logo_image_path,output_image_path)